In [1]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf

env = gym.make("FrozenLake-v0")

hdf5 not supported (please install/reinstall h5py)


[2017-01-26 15:13:34,801] Making new env: FrozenLake-v0


In [2]:
def next_action(actions, e):
    n = len(actions)
    
    if random.random() < e:
        return random.randint(0, n-1)
    else:
        return np.argmax(actions)
    
def select_columns(tensor, indexes):
    idx = tf.stack((tf.range(tf.shape(indexes)[0]), indexes), 1)
    return tf.gather_nd(tensor, idx)

In [7]:
n_actions = env.action_space.n
n_states = env.observation_space.n
model_name = "deep-q.model"
model_path = "/models/" + model_name
y = 0.95

graph = tf.Graph()
with graph.as_default():
    with tf.device("cpu:0"):
        S = tf.placeholder(tf.int32, [None], name='s')
        A = tf.placeholder(tf.int32, [None], name='a')
        R = tf.placeholder(tf.float32, [None], name='r')
        MaxQS1 = tf.placeholder(tf.float32, [None], name='maxQs1')
        LR = tf.placeholder(tf.float32, [], name='lr')

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        QS = Pipe(
            S,
            T.one_hot(n_states)
            .relu_layer(32, **ops)
            .linear_layer(4, scope='linear_layer', **ops),
            T
        )
        
        QSA = select_columns(QS, A)
        
        MaxQS = tf.reduce_max(QS, 1)
        
        E = R + y * MaxQS1 - QSA
        Loss = Pipe(E, tf.nn.l2_loss, tf.reduce_sum)
        Update = tf.train.GradientDescentOptimizer(LR).minimize(Loss)

        Writer = tf.summary.FileWriter('/logs/' +  model_name)
        Saver = tf.train.Saver()

In [8]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device("cpu:0"):
        sess.run(tf.global_variables_initializer())
    #     saver.restore(sess, model_path)

        r_total = 0
        k = 2000.0

        for t in range(200000):
            lr = 0.2 * k / (k + t)

            s = env.reset()
            qs = sess.run(QS, feed_dict={S: [s]})[0]

            while True:
                #next action
                a = next_action(qs, 0.1)

                #take step
                s1, r, done, info = env.step(a)
                r_total += r

                #calculate next Qs
                [maxQs1, qs1] = sess.run([MaxQS, QS], feed_dict={S: [s1]})

                #train
                sess.run(Update, feed_dict={
                    S: [s], A: [a], R: [r], 
                    LR: lr,
                    MaxQS1: np.max(qs1, 1)
                })

                #update state
                s = s1
                qs = qs1[0]

                if done:
                    break

            if t % 500 == 0:
                print r_total, "of", 500, ", lr:", lr
                r_total = 0
                Saver.save(sess, model_path)


0.0 of 500 , lr: 0.2
0.0 of 500 , lr: 0.16
13.0 of 500 , lr: 0.133333333333
17.0 of 500 , lr: 0.114285714286
18.0 of 500 , lr: 0.1
15.0 of 500 , lr: 0.0888888888889
15.0 of 500 , lr: 0.08
23.0 of 500 , lr: 0.0727272727273
10.0 of 500 , lr: 0.0666666666667
11.0 of 500 , lr: 0.0615384615385
17.0 of 500 , lr: 0.0571428571429


KeyboardInterrupt: 

In [ ]:
s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    Saver.restore(sess, model_path)
    
    for i in range(100):
        qs = sess.run(QS, feed_dict={S: [s]})
        a = next_action(qs, 0)
        s, r, done, info = env.step(a)
        env.render()
        print("")

        if done:
            print(r)
            break